In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model
from tensorflow.keras import backend as k
import numpy as np
import json
import os
import gc
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

In [2]:
publaynet_data = np.load("C://Users//kishore prashanth//Downloads//ValidRoomIdsnumpyXcrted.npy",allow_pickle=True)

In [3]:
publaynet_data.shape

(13315, 9, 5)

In [4]:
class MMHSALayer(Layer):
    def __init__(self,heads=8):
        super(MMHSALayer, self).__init__()
        self.heads = heads

    def build(self,input_shape):
        #print("MMSHALAYER build:",input_shape)
        self.model_dim = input_shape[-2]
        self.k = self.add_weight(shape=(self.heads,self.model_dim,self.model_dim),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="Key")
        self.q = self.add_weight(shape=(self.heads,self.model_dim,self.model_dim),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="Query")
        self.v = self.add_weight(shape=(self.heads,self.model_dim,self.model_dim),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="Value")
        self.o = self.add_weight(shape=(self.model_dim,self.model_dim*self.heads),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="Heads")

    def call(self,inputs):
        #print("MMSHALAYER call:",inputs.shape)
        mask_shape = inputs.shape[-1]

        mask_0 = np.ones((mask_shape,mask_shape))
        for i in range(mask_shape):
            for j in range(mask_shape):
                if (i>j):
                    mask_0[i][j]=0
        self.mask_0 = tf.constant(mask_0,dtype=tf.float32)

        mask_inf = np.zeros((mask_shape,mask_shape))
        for i in range(mask_shape):
            for j in range(mask_shape):
                if (i>j):
                    mask_inf[i][j]=-10000000000
        self.mask_inf = tf.constant(mask_inf,dtype=tf.float32)

        inputs = tf.expand_dims(inputs,1)

        key=tf.matmul(self.k,inputs)
        que=tf.matmul(self.q,inputs)
        val=tf.matmul(self.v,inputs)

        Z=tf.matmul(tf.transpose(key,perm=[0,1,3,2]),que)*(1/np.sqrt(self.model_dim))
        W=tf.multiply(Z,self.mask_0)
        W=tf.add(W,self.mask_inf)
        W=tf.keras.activations.softmax(W,axis=1)
        W=tf.multiply(W,self.mask_0)
        W=tf.matmul(val,W)

        W = tf.reshape(W,(inputs.shape[0],self.model_dim*self.heads,mask_shape))

        ans = W

        ans = tf.matmul(self.o,ans)
        ans=tf.expand_dims(ans,0)

        ans = tf.squeeze(ans,axis=0)

        return ans

class Dense2D(Layer):
    def __init__(self,units):
        super(Dense2D, self).__init__()
        self.units = units

    def build(self,input_shape):
        #print("Dense2D build:",input_shape)
        input_len = input_shape[-2]

        self.w = self.add_weight(shape=(self.units,input_len),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="dense2dw")

    def call(self,inputs):
        #print("Dense2D call:",inputs)
        ans = tf.matmul(self.w,inputs)

        return ans

class FFLayer(Layer):
    def __init__(self, dff=2048, dropout=0.1):
        super(FFLayer,self).__init__()
        self.dff = dff
        self.dropout = dropout

    def build(self,input_shape):
        #print("FFLayer build:",input_shape)
        self.dropout = tf.keras.layers.Dropout(self.dropout)
        self.dout = input_shape[-2]

        self.w1 = self.add_weight(shape=(self.dff,self.dout),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="ffw1")
        self.w2 = self.add_weight(shape=(self.dout,self.dff),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="ffw2")
        self.b1 = self.add_weight(shape=(self.dff,1),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="ffb1")
        self.b2 = self.add_weight(shape=(self.dout,1),
                                 initializer='random_normal',
                                 trainable=True,
                                 name="ffb2")

    def call(self,inputs):
        #print("FFLayer call:",inputs.shape)
        ans = tf.add(tf.matmul(self.w1,inputs),self.b1)
        ans = tf.keras.activations.relu(ans)
        ans = tf.add(tf.matmul(self.w2,ans),self.b2)

        ans = self.dropout(ans)

        return ans

class ANLayer(Layer):
    def __init__(self):
        super(ANLayer,self).__init__()
        self.Normal = tf.keras.layers.LayerNormalization(axis=1)

    def call(self,inputs1,inputs2):
        #print("ANlayer call:",inputs1.shape)
        #print("ANLayer call:",inputs2.shape)
        sum = tf.add(inputs1,inputs2)
        ans=self.Normal(sum)
        return ans

In [5]:
class LTModel(Model):
    def __init__(self, input_shape, layers, heads, dff, model_dim, dropout):
        super(LTModel, self).__init__()

        self.emb = Dense2D(model_dim)

        self.SA = []
        self.AN1 = []
        self.FF = []
        self.AN2 = []

        for i in range(layers):
            self.SA.append(MMHSALayer(heads))
            self.AN1.append(ANLayer())
            self.FF.append(FFLayer(dff, dropout))
            self.AN2.append(ANLayer())

        self.deemb = Dense2D(input_shape)
        self.sm = tf.keras.layers.Softmax(axis=1)

    def call(self, x):
        x = self.emb(x)

        for i in range(len(self.SA)):
            y = self.SA[i](x)
            x = self.AN1[i](x,y)
            y = self.FF[i](x)
            x = self.AN2[i](x,y)

        x = self.deemb(x)
        x = self.sm(x)

        return x

In [6]:
class LayoutTransformer:

    def __init__(self, n_classes, class_labels=None, n_anchors=(32,32), d=512, n_layers=6, n_heads=8, dff=2048, dropout=0.1):
        self.n_classes = n_classes+2
        self.n_anchors = n_anchors
        self.d = d
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.dff = dff
        self.dropout = dropout
        self.n_row = n_anchors[0]
        self.n_col = n_anchors[1]
        self.input_dim = 2+n_classes+2*(n_anchors[0]+n_anchors[1])
        self.model = LTModel(self.input_dim, model_dim=d, layers=n_layers, heads=n_heads, dff=dff, dropout=dropout)
        self.loss_his = []
        self.lr_his = []
        self.train_data_his = []
        if class_labels == None:
            self.labels = range(1,n_classes+1)
        else:
            self.labels = class_labels
        self.x_data=[]

    def compile(self, lr=1e-5):
        self.model.compile(loss=tf.keras.losses.KLDivergence(),
                           metrics = [tf.keras.losses.KLDivergence()],
                           optimizer = tf.keras.optimizers.Adam(learning_rate=lr))

    def build(self):
        self.model.build((1,self.input_dim,1))
    
    def train(self, epochs, batch_size=1, train_data_index="All", rlrop_factor=0.5, rlrop_patience=1000, rlrop_min_delta=0.001):
        if train_data_index == "All":
            train_data_index = range(self.data.shape[0])
        rlrop = tf.keras.callbacks.ReduceLROnPlateau(factor=rlrop_factor,patience=rlrop_patience,verbose=1,min_delta=rlrop_min_delta,monitor='kl_divergence')
        callbacks = [rlrop]
        history = self.model.fit(x=tf.convert_to_tensor(self.x_data[train_data_index]), y=tf.convert_to_tensor(self.y_data[train_data_index]), epochs=epochs, batch_size=batch_size, callbacks=callbacks)
        self.loss_his.extend(history.history['loss'])
        self.lr_his.extend(history.history['lr'])
        for i in range(epochs):
            self.train_data_his.append(len(train_data_index))

    def load_weights(self, folder_path, filename):
        self.build()
        self.model.load_weights(folder_path + '/' + str(filename) + '.h5')
        his = json.loads(open(folder_path + '/' + str(filename) + '.json').read())

        self.loss_his = his['loss']
        self.train_data_his = his['data']
        self.lr_his = his['lr']


    def save_weights(self, folder_path, filename):
        his = json.dumps({'loss':list(np.array(self.loss_his,dtype='float')),'data':list(np.array(self.train_data_his,dtype='float')),'lr':list(np.array(self.lr_his,dtype='float'))})
        open(folder_path + '/' + str(filename) + '.json','w').write(his)
        self.model.save_weights(folder_path + '/' + str(filename) + '.h5')

    def load_data(self, data, rows, cols, e=0.1):

        self.orig_data = np.array(data,dtype='float32')
        data = np.array(data,dtype='float32')

        data[:,:,1] = data[:,:,1]*(cols*(self.n_col-1)) 
        data[:,:,2] = data[:,:,2]*(rows*(self.n_row-1))
        data[:,:,3] = data[:,:,3]*(cols*(self.n_col-1))
        data[:,:,4] = data[:,:,4]*(rows*(self.n_row-1))

        data = np.array(data,dtype='int')

        # Sorting
        for i in range(data.shape[0]):
            box_num = data[i].shape[0]

            c=0
            for j in data[i]:
                if j[3]==0 and j[4]==0:
                    break
                c = c+1 #total number of valid boxes

            order = [*list(data[i][0:c,3].argsort()),*range(c,box_num)] # 4 Width (Col)
            data[i] = np.array(data[i,order])
            order = [*list(data[i][0:c,4].argsort()),*range(c,box_num)] # 3 Height (Row)
            data[i] = np.array(data[i,order])
            order = [*list(data[i][0:c,1].argsort()),*range(c,box_num)] # 2 X-Pos (Col)
            data[i] = np.array(data[i,order])
            order = [*list(data[i][0:c,2].argsort()),*range(c,box_num)] # 1 Y-Pos (Row)
            data[i] = np.array(data[i,order])

        self.data = data

        # One hot encoding
        onehot_data = []

        for doc in data:
            cur_data = []
            for box in doc:
                cur_cur_data = list(np.zeros(self.input_dim))
                cur_cur_data[box[0]] = 1
                cur_cur_data[box[1]+self.n_classes] = 1
                cur_cur_data[box[2]+self.n_classes+self.n_col] = 1
                cur_cur_data[box[3]+self.n_classes+self.n_col+self.n_row] = 1
                cur_cur_data[box[4]+self.n_classes+self.n_col*2+self.n_row] = 1
                cur_data.append(cur_cur_data)
            onehot_data.append(cur_data)

        self.onehot_data = np.array(onehot_data, dtype='float32') ######

        # x_data with <bos> and y_data with <eos>
        x_data = []
        y_data = []

        for doc in onehot_data:
            bos = list(np.zeros(self.input_dim))
            bos[0]=1
            x = [bos,*doc]
            x = np.array(x).T
            x_data.append(x)

            eos = list(np.zeros(self.input_dim))
            eos[self.n_classes-1] = 1
            y = [*doc,eos]
            for box in y:
                for k in range(0, self.n_classes):
                    box[k] = (1 - e) * box[k] + e / self.n_classes
                for k in range(self.n_classes, self.n_classes+self.n_col):
                    box[k] = (1 - e) * box[k] + e / self.n_col
                for k in range(self.n_classes+self.n_col, self.n_classes+self.n_col+self.n_row):
                    box[k] = (1 - e) * box[k] + e / self.n_row
                for k in range(self.n_classes+self.n_col+self.n_row, self.n_classes+2*self.n_col+self.n_row):
                    box[k] = (1 - e) * box[k] + e / self.n_col
                for k in range(self.n_classes+2*self.n_col+self.n_row, self.n_classes+2*self.n_col+2*self.n_row):
                    box[k] = (1 - e) * box[k] + e / self.n_row
            y = np.array(y).T
            y_data.append(y)

        self.x_data = np.array(x_data,dtype="float32")
        self.y_data = np.array(y_data,dtype="float32")


In [7]:
publay_model = LayoutTransformer(n_classes=11, class_labels=["None","Living room","Master room","Kitchen","Bathroom","Dining room",
              "Child room","Study room","Second room","Guest room","Balcony"])

In [8]:
publay_model.load_data(publaynet_data[0:13315],rows=1,cols=1)

In [ ]:
model = publay_model

epochs = 1
lrate = 1e-5

min_delta = 0.001
patience = 20
factor = 0.95

count = 0
model.compile(lr=lrate)

for i in range(epochs):
    gc.collect()
    k.clear_session()
    try:
        if model.loss_his[-2]-model.loss_his[-1]<min_delta:
            count = count+1
    except:
        pass

    if count==patience:
        count = 0
        lrate = lrate*factor
        model.compile(lr=lrate)

    model.train(epochs=1, batch_size=1, train_data_index='All')
    if i>0 and i%5==0:
      model.save_weights('./','model'+str(i+1))